# C3 FC percentile validation

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with the`DEA Sandbox` environments
* **Products used:** 
[fc_percentile_albers_annual](https://explorer.sandbox.dea.ga.gov.au/products/fc_percentile_albers_annual), 
C3 fc percentile test product

## Description
The notebook is to validate the new C3 fc percentile product against the C2 product `fc_percentile_albers_annual`. It produced the output for the validation report.

1. Generate distritubtions and plot PDFs as the validation results
2. Produce the summary of validation results
3. Plot examples of the findings

***

## Getting started

Install the package needed by

`!pip install awswrangler`

in the top cell or the terminal then restart notebook.

In [ ]:
import datacube
import rasterio
import boto3
import xarray as xr
import numpy as np
import re
from datacube.utils.dask import start_local_dask
from datacube import Datacube
from datacube.utils.geometry import CRS, Geometry, GeoBox
from osgeo import ogr, gdal, osr
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy.stats as sps
import awswrangler as wr
import pandas as pd
import fiona

from odc.algo.io import load_with_native_transform
from odc.algo import keep_good_only
from odc.algo._masking import _xr_fuse, _or_fuser, _fuse_mean_np, _fuse_or_np, _fuse_and_np, enum_to_bool
from odc.stats.utils import fuse_ds, fuse_products
from functools import partial
from itertools import groupby

from scipy.stats import linregress

In [ ]:
# create a local cluster
client = start_local_dask(n_workers=1, threads_per_worker=60, memory_limit='478GB')
client

In [ ]:
# `dev` is the credential profile name
# change it accordingly
session = boto3.Session(profile_name='dev')
fc_bucket = "s3://dea-public-data-dev/test/fc-percentile/"

In [ ]:
grid_list = pd.read_csv("../../../grid_test_fc_percentile_mangroves.csv")
test_grids = grid_list["New grid"]
dc = Datacube()

In [ ]:
def poly_from_region_code(region_code, grids_file):
    with fiona.open(grids_file) as allshapes:
        for shape in allshapes:
            if shape['properties'].get('region_code', '') == region_code.lower():
                return Geometry(shape['geometry'], crs=CRS('EPSG:4326'))
    
def generate_seamask(shape_file, data_shape, orig_coords, resolution):
    """
        creak mask without oceans
        input:
            shape_file: the shape file of Australia coastline
            data_shape: the shape of loaded data to be masked upon
            orig_coords: the origin of the image for gdal to decide the transform
            resolution: pixel size with signs, e.g., (30, -30) for C3 and (25, -25) for C2
        output:
            a numpy array of mask, where valid pixels = 1
    """
    source_ds = ogr.Open(shape_file)
    source_layer = source_ds.GetLayer()
    source_layer.SetAttributeFilter("FEAT_CODE!='sea'")

    yt, xt = data_shape
    xres = resolution[0]
    yres = resolution[1]
    no_data = 0

    xcoord, ycoord = orig_coords
    geotransform = (xcoord - (xres*0.5), xres, 0, ycoord - (yres*0.5), 0, yres)

    target_ds = gdal.GetDriverByName('MEM').Create('', xt, yt, gdal.GDT_Byte)
    target_ds.SetGeoTransform(geotransform)
    albers = osr.SpatialReference()
    albers.ImportFromEPSG(3577)
    target_ds.SetProjection(albers.ExportToWkt())
    band = target_ds.GetRasterBand(1)
    band.SetNoDataValue(no_data)

    gdal.RasterizeLayer(target_ds, [1], source_layer, burn_values=[1])
    return band.ReadAsArray()


In [ ]:
    def _native_tr(xx):
        """
        Loads data in its native projection. It performs the following:

        1. Load all fc and WOfS bands
        2. Set the high terrain slope flag to 0
        3. Set all pixels that are not clear and dry to NODATA
        4. Calculate the clear wet pixels
        5. Drop the WOfS band
        """
        water = xx.water & 0b1110_1111
        dry = water == 0
        xx = xx.drop_vars(["water"])
        xx = keep_good_only(xx, dry, nodata=255)
        return xx

    def _fuser(xx):
        xx = _xr_fuse(xx, partial(_fuse_mean_np, nodata=255), '')
        return xx
    
    def filter(groups, size=2):
        for _, ds_group in groups:
            ds_group = tuple(ds_group)
            if len(ds_group) == size:
                yield ds_group
    
    def ds_align(datasets):
        datasets.sort(key=lambda ds: (ds.center_time, ds.metadata.region_code))
        paired_dss = groupby(datasets, key=lambda ds: (ds.center_time, ds.metadata.region_code))
        paired_dss = filter(paired_dss)
        map_fuse_func = lambda x: fuse_ds(*x)
        dss = map(map_fuse_func, paired_dss)
        return dss
    
    def _fuser_nbart(xx):
        xx = _xr_fuse(xx, partial(_fuse_mean_np, nodata=-999), '')
        return xx
    
    def _native_tr_nbart(xx):
        """
        Loads data in its native projection.
        """
        bad = enum_to_bool(xx["fmask"], ("nodata", "cloud", "shadow", "water")) # a pixel is bad if any of the cloud, shadow, or no-data value
        bad |= xx["nbart_contiguity"] == 0 # or the nbart contiguity bit is 0
        xx = xx.drop_vars(["fmask", "nbart_contiguity"])
        
        for band in xx.data_vars.keys():
            bad = bad | (xx[band] == -999)
        xx = keep_good_only(xx, ~bad, nodata=-999)
        return xx

In [ ]:
def plot_yearly_diff(grid, ls8_data, ls7_data, years, band, collection):
    data_to_plot = []
    fig, axs = plt.subplots(1, 1,  sharey=True, sharex=True, figsize=(8, 8))
    pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
    for p in pcs:
        data_to_plot = []
        for y in range(years[0], years[1]):
            data_to_plot += [ls8_data[band].loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')].median() - 
                             ls7_data[band].loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')].median()]
        axs.plot(np.arange(years[0], years[1]), data_to_plot, label="Median difference on %s percentile" %  int(p*100))
    axs.axhline(y=0, color='black', linestyle='dashdot')
    plt.tight_layout()
    fig.legend(loc='upper left', ncol=3)
    plt.savefig('fc_diff_plot/' + '_'.join([grid, band, collection]) + '_yearly_median_diff.png', bbox_inches='tight')
    plt.close()

In [ ]:
def plot_alltime_box(grid, ls8_data, ls7_data, band, collection):
    fig, axs= plt.subplots(figsize=(10, 6))
    pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
    positions = np.arange(1, (len(pcs) + 1), 1.0)
    labels_ls7 = ['LS7-'+str(int(p*100)) for p in pcs]
    data_to_plot = []
    for p in pcs:
        data_to_plot += [ls7_data[band].loc[p]]
    box_plot_ls7 = axs.boxplot(data_to_plot, vert=1, widths=0.3, patch_artist=True, showfliers=False,
                  positions=positions,
                  labels=labels_ls7)
    
    positions += 0.4
    labels_ls8 = ['LS8-'+str(int(p*100)) for p in pcs]
    data_to_plot = []
    for p in pcs:
        data_to_plot += [ls8_data[band].loc[p]]
    box_plot_ls8 = axs.boxplot(data_to_plot, vert=1, widths=0.3, showfliers=False,
                  positions=positions,
                  labels=labels_ls8)
    axs.set_xticklabels(labels_ls7+labels_ls8,
                    rotation=45, fontsize=8)
    for item in ['boxes', 'whiskers', 'caps']:
        plt.setp(box_plot_ls7[item], color='darkblue')
    plt.savefig('fc_diff_plot/' + '_'.join([grid, band, collection]) + '_alltime_diff.png', bbox_inches='tight')
    plt.close()

In [ ]:
chunks = {"y": -1, "x": -1}
first = False
pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
for grid in test_grids:
    print("grid", grid)
    if grid.lower() != 'x39y13' and grid.lower() != 'x40y13':
        continue
    # if grid.lower() != 'x35y22' and not first:
    #    continue
    # if grid.lower() == 'x35y22':
    #    first = True
    query_poly = poly_from_region_code(grid, "../../../au-grid.geojson")
    c3_query = {'geopolygon': query_poly}
    c3_query['time'] = ('2014-01-01', '2021-01-01')
    geobox = GeoBox.from_geopolygon(query_poly, (-30, 30), crs='epsg:3577')
    
    c3_ls7_datasets = dc.find_datasets(product=['ga_ls_wo_3', 'ga_ls_fc_3'], **c3_query,
                 platform="landsat-7", group_by="solar_day")
    c3_ls7_datasets = ds_align(c3_ls7_datasets)
    c3_ls7 = load_with_native_transform(
        c3_ls7_datasets,
        bands=["water", "pv", "bs", "npv"],
        geobox=geobox,
        native_transform=_native_tr,
        fuser=_fuser,
        groupby="solar_day",
        resampling="bilinear",
        chunks=chunks,
    )
    c3_land_raster = generate_seamask("../../../aus_map/cstauscd_r_3577.shp",
                                      c3_ls7.pv.shape[1:], (c3_ls7.x.data.min(), c3_ls7.y.data.max()), (30, -30))

    c3_ls8_datasets = dc.find_datasets(product=['ga_ls_fc_3', 'ga_ls_wo_3'], **c3_query,
                 platform="landsat-8", group_by="solar_day")
    c3_ls8_datasets = ds_align(c3_ls8_datasets)
    c3_ls8 = load_with_native_transform(
        c3_ls8_datasets,
        bands=["water", "pv", "bs", "npv"],
        geobox=geobox,
        native_transform=_native_tr,
        fuser=_fuser,
        groupby="solar_day",
        resampling="bilinear",
        chunks=chunks,
    )
    c3_ls7 = c3_ls7.where((c3_ls7 < 255) & c3_land_raster)
    c3_ls8 = c3_ls8.where((c3_ls8 < 255) & c3_land_raster)
    ls7_pc_10 = c3_ls7.quantile(pcs, dim=['x', 'y'], skipna=True).compute().dropna('spec', how='all')
    ls8_pc_10 = c3_ls8.quantile(pcs, dim=['x', 'y'], skipna=True).compute().dropna('spec', how='all')
    ls7_pc_10 = ls7_pc_10.reset_index(['time', 'idx', 'uuid', 'grid'], drop=True).rename({'spec': 'time'}).to_dataframe()
    ls8_pc_10 = ls8_pc_10.reset_index(['time', 'idx', 'uuid', 'grid'], drop=True).rename({'spec': 'time'}).to_dataframe()
    ls7_pc_10.to_csv(grid.lower() + '_fc_ls7_c3.csv')
    ls8_pc_10.to_csv(grid.lower() + '_fc_ls8_c3.csv')

In [ ]:
for grid in test_grids:
    grid = grid.lower()
    ls8_pc_10 = pd.read_csv('fc_diff_data/'+grid+'_fc_ls8_c3.csv')
    ls8_pc_10['time'] = ls8_pc_10['time'].astype(np.datetime64)
    ls8_pc_10 = ls8_pc_10.set_index(['quantile', 'time'])
    ls7_pc_10 = pd.read_csv('fc_diff_data/'+grid+'_fc_ls7_c3.csv')
    ls7_pc_10['time'] = ls7_pc_10['time'].astype(np.datetime64)
    ls7_pc_10 = ls7_pc_10.set_index(['quantile', 'time'])
    plot_yearly_diff(grid, ls8_pc_10, ls7_pc_10, (2014, 2021), 'pv', 'c3')
    plot_alltime_box(grid, ls8_pc_10, ls7_pc_10,'pv', 'c3')

In [ ]:
max_median_ls8 = []
max_median_ls7 = []
pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
for grid in test_grids:
    grid = grid.lower()
    ls8_pc_10 = pd.read_csv('fc_diff_data/'+grid+'_fc_ls8_c3.csv')
    ls8_pc_10['time'] = ls8_pc_10['time'].astype(np.datetime64)
    ls8_pc_10 = ls8_pc_10.set_index(['quantile', 'time'])
    ls7_pc_10 = pd.read_csv('fc_diff_data/'+grid+'_fc_ls7_c3.csv')
    ls7_pc_10['time'] = ls7_pc_10['time'].astype(np.datetime64)
    ls7_pc_10 = ls7_pc_10.set_index(['quantile', 'time'])
    for p in pcs:
        max_median_ls8 += [ls8_pc_10.loc[p].pv.median()]
        max_median_ls7 += [ls7_pc_10.loc[p].pv.median()]

In [ ]:
fc_lin = linregress(max_median_ls8, max_median_ls7)

In [ ]:
fig, axs = plt.subplots(1, 1,  sharey=True, sharex=True, figsize=(9, 6))
axs.plot(max_median_ls8, max_median_ls7, 'o',  color='SteelBlue',  mfc='none', markersize=5)
axs.set_xlabel("LS8")
axs.set_ylabel("LS7")
axs.axline([0, 0], [1, 1], color='darkgreen', ls='--', label="1:1")
axs.plot(max_median_ls8, fc_lin.slope*np.array(max_median_ls8)+fc_lin.intercept, color='darkorange', label='regression')
handles, labels = axs.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', ncol=1)
plt.tight_layout()
plt.savefig('fc_diff_plot/' + 'fc_regression.png', bbox_inches='tight')

In [ ]:
test_grids[np.argsort(max_median)]

In [ ]:
fig, axs = plt.subplots(3, 3,  sharey=True, sharex=True, figsize=(15, 15))
i = 0
j = 0
year = 2014
pcs = [0.2, 0.5, 0.8]
for y in range(2014, 2021):
    data_to_plot = []
    for p in pcs:
        data_to_plot += [ls7_pc.pv.loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')],
                         ls8_pc.pv.loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')]]
    box_plot = axs[i, j].boxplot(data_to_plot,
                  positions=[1, 1.6, 2.5, 3.1, 4, 4.6],
                  labels=['LS7-20','LS8-20','LS7-50','LS8-50','LS7-80','LS8-80'])
    axs[i, j].set_title(str(y))
    if j >= 2:
        i += 1
        j = 0
    else:
        j += 1
plt.savefig(test_grids[0] + '_pv_c3_yearly_diff.png', bbox_inches='tight')

In [ ]:
for grid in test_grids:
    query_poly = poly_from_region_code(grid, "../../../au-grid.geojson")
    c3_query = {'geopolygon': query_poly}
    c3_query['time'] = ('2014-01-01', '2020-01-01')
    c2_ls7 = dc.load(product=['ls7_fc_albers'], **c3_query, group_by="solar_day", measurements=['BS', 'PV', 'NPV'], 
                     dask_chunks={'time':1})
    c2_wofs = dc.load(product=['wofs_albers'], **c3_query, group_by="solar_day", platform='LANDSAT_7', measurements=['water'],
                      dask_chunks={'time':1})
    c2_land_raster = generate_seamask("../../../aus_map/cstauscd_r_3577.shp", c2_ls7.PV.shape[1:],
                                      (c2_ls7.x.data.min(), c2_ls7.y.data.max()), (25, -25))
    dates = np.intersect1d(c2_ls7.time.data, c2_wofs.time.data)
    water = c2_wofs.water & 0b1110_1011
    dry = water == 0
    c2_ls7 = c2_ls7.loc[dict(time=dates)]
    dry = dry.loc[dict(time=dates)]
    c2_ls7 = c2_ls7.where((c2_ls7 > -1) & dry & c2_land_raster)

    c2_ls8 = dc.load(product=['ls8_fc_albers'], **c3_query, group_by="solar_day", measurements=['BS', 'PV', 'NPV'], 
                     dask_chunks={'time':1})
    c2_wofs = dc.load(product=['wofs_albers'], **c3_query, group_by="solar_day", platform='LANDSAT_8', measurements=['water'],
                      dask_chunks={'time':1})
    dates = np.intersect1d(c2_ls8.time.data, c2_wofs.time.data)
    water = c2_wofs.water & 0b1110_1011
    dry = water == 0
    c2_ls8 = c2_ls8.loc[dict(time=dates)]
    dry = dry.loc[dict(time=dates)]
    c2_ls8 = c2_ls8.where((c2_ls8 > -1) & dry & c2_land_raster)

    ls7_pc_10 = c2_ls7.quantile([0.2, 0.5, 0.8], dim=['x', 'y'], skipna=True).compute().dropna('time', how='all')
    ls8_pc_10 = c2_ls8.quantile([0.2, 0.5, 0.8], dim=['x', 'y'], skipna=True).compute().dropna('time', how='all')
    ls7_pc_10 = ls7_pc_10.to_dataframe()
    ls8_pc_10 = ls8_pc_10.to_dataframe()
    ls7_pc_10.to_csv(grid.lower() + '_fc_ls7_c2.csv')
    ls8_pc_10.to_csv(grid.lower() + '_fc_ls8_c2.csv')

In [ ]:
fig, axs = plt.subplots(3, 3,  sharey=True, sharex=True, figsize=(15, 15))
i = 0
j = 0
year = 2014
pcs = [0.2, 0.5, 0.8]
for y in range(2014, 2021):
    data_to_plot = []
    for p in pcs:
        data_to_plot += [ls7_pc_c2.PV.loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')],
                         ls8_pc_c2.PV.loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')]]
    box_plot = axs[i, j].boxplot(data_to_plot,
                  positions=[1, 1.6, 2.5, 3.1, 4, 4.6],
                  labels=['LS7-20','LS8-20','LS7-50','LS8-50','LS7-80','LS8-80'])
    axs[i, j].set_title(str(y))
    if j >= 2:
        i += 1
        j = 0
    else:
        j += 1
plt.savefig(test_grids[0] +'_pv_c2_yearly_diff.png', bbox_inches='tight')

In [ ]:
chunks = {"y": -1, "x": -1}

for grid in test_grids:
    query_poly = poly_from_region_code(grid, "../../../au-grid.geojson")
    c3_query = {'geopolygon': query_poly}
    c3_query['time'] = ('2014-01-01', '2021-01-01')
    geobox = GeoBox.from_geopolygon(query_poly, (-30, 30), crs='epsg:3577')
    
    c3_ls7_datasets = dc.find_datasets(product=['ga_ls7e_ard_3'], **c3_query)
    c3_ls7 = load_with_native_transform(
        c3_ls7_datasets,
        bands=["blue", "green", "red", "nir", "swir1", "swir2", "fmask", "nbart_contiguity"],
        geobox=geobox,
        native_transform=_native_tr_nbart,
        fuser=_fuser_nbart,
        groupby="solar_day",
        resampling="nearest",
        chunks=chunks,
    )
    c3_land_raster = generate_seamask("../../../aus_map/cstauscd_r_3577.shp",
                                          c3_ls7.blue.shape[1:], (c3_ls7.x.data.min(), c3_ls7.y.data.max()), (30, -30))
    
    c3_ls8_datasets = dc.find_datasets(product='ga_ls8c_ard_3', **c3_query)
    c3_ls8 = load_with_native_transform(
        c3_ls8_datasets,
        bands=["blue", "green", "red", "nir", "swir1", "swir2", "fmask", "nbart_contiguity"],
        geobox=geobox,
        native_transform=_native_tr_nbart,
        fuser=_fuser_nbart,
        groupby="solar_day",
        resampling="nearest",
        chunks=chunks,
    )
    c3_ls7 = c3_ls7.where((c3_ls7 > -999) & c3_land_raster)
    c3_ls8 = c3_ls8.where((c3_ls8 > -999) & c3_land_raster)
    ls7_pc_10 = c3_ls7.quantile(pcs, dim=['x', 'y'], skipna=True).compute().dropna('spec', how='all')
    ls8_pc_10 = c3_ls8.quantile(pcs, dim=['x', 'y'], skipna=True).compute().dropna('spec', how='all')
    ls7_pc_10 = ls7_pc_10.reset_index(['time', 'idx', 'uuid', 'grid'], drop=True).rename({'spec': 'time'}).to_dataframe()
    ls8_pc_10 = ls8_pc_10.reset_index(['time', 'idx', 'uuid', 'grid'], drop=True).rename({'spec': 'time'}).to_dataframe()
    ls7_pc_10.to_csv(grid.lower() + '_nbart_ls7_c3.csv')
    ls8_pc_10.to_csv(grid.lower() + '_nbart_ls8_c3.csv')

In [ ]:
fig, axs = plt.subplots(3, 3,  sharey=True, sharex=True, figsize=(15, 15))
i = 0
j = 0
year = 2014
for a, b in zip(ls7_pc, ls8_pc):
    data_to_plot = []
    for _a, _b in zip(a.swir1, b.swir1):
        data_to_plot += [_a, _b]
    box_plot = axs[i, j].boxplot(data_to_plot, showfliers=False,
                  positions=[1, 1.6, 2.5, 3.1, 4, 4.6],
                  labels=['LS7-20','LS8-20','LS7-50','LS8-50','LS7-80','LS8-80'])
    axs[i, j].set_title(str(year))
    year += 1
    if j >= 2:
        i += 1
        j = 0
    else:
        j += 1

In [ ]:
sf_bands = ["blue", "green", "red", "nir", "swir1", "swir2"]
pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
for grid in test_grids:
    grid = grid.lower()
    ls8_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls8_c3.csv')
    ls8_pc['time'] = ls8_pc['time'].astype(np.datetime64)
    ls8_pc = ls8_pc.set_index(['quantile', 'time'])
    ls7_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls7_c3.csv')
    ls7_pc['time'] = ls7_pc['time'].astype(np.datetime64)
    ls7_pc = ls7_pc.set_index(['quantile', 'time'])
    fig, axs = plt.subplots(2, 3,  sharey=True, sharex=True, figsize=(15, 10))
    data_to_plot = []
    i = 0
    j = 0
    for band in sf_bands:
        for p in pcs:
            data_to_plot = []
            for y in range(2014, 2021):
                data_to_plot += [ls8_pc[band].loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')].median() - 
                                 ls7_pc[band].loc[p].loc[slice(str(y)+'-01-01', str(y+1)+'-01-01')].median()]
            axs[j, i].plot(np.arange(2014, 2021), data_to_plot, label="Median difference on %s percentile" %  int(p*100))
        axs[j, i].axhline(y=0, color='black', linestyle='dashdot')
        axs[j, i].set_title(band)
        if i >= 2:
            j += 1
            i = 0
        else:
            i += 1
    plt.tight_layout()
    handles, labels = axs[0, 0].get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper left', ncol=3)
    plt.savefig('nbart_diff_plot/' + grid +'_nbar_c3_yearly_median_diff.png', bbox_inches='tight')
    plt.close()

In [ ]:
for band in sf_bands:
    for q in range(3):
        data_to_plot = []
        for a, b in zip(ls7_pc, ls8_pc):
            data_to_plot += [np.median(b[band].data[q]) - np.median(a[band].data[q])]
        axs[j, i].plot(np.arange(2014, 2021), data_to_plot, label="Median difference on %s percentile" %  pcs[q])

In [ ]:
pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
for grid in test_grids:
    grid = grid.lower()
    ls8_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls8_c3.csv')
    ls8_pc['time'] = ls8_pc['time'].astype(np.datetime64)
    ls8_pc = ls8_pc.set_index(['quantile', 'time'])
    ls7_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls7_c3.csv')
    ls7_pc['time'] = ls7_pc['time'].astype(np.datetime64)
    ls7_pc = ls7_pc.set_index(['quantile', 'time'])
    fig, axs = plt.subplots(2, 3,  sharey=True, sharex=True, figsize=(15, 10))
    i = 0
    j = 0
    for band in sf_bands:
        pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
        positions = np.arange(1, (len(pcs) + 1), 1.0)
        labels_ls7 = ['LS7-'+str(int(p*100)) for p in pcs]
        data_to_plot = []
        for p in pcs:
            data_to_plot += [ls7_pc[band].loc[p]]
        box_plot_ls7 = axs[j, i].boxplot(data_to_plot, vert=1, widths=0.3, patch_artist=True, showfliers=False,
                      positions=positions,
                      labels=labels_ls7)

        axs[j, i].set_xticks(list(positions) + list(positions+0.4))
        positions += 0.4
        labels_ls8 = ['LS8-'+str(int(p*100)) for p in pcs]
        data_to_plot = []
        for p in pcs:
            data_to_plot += [ls8_pc[band].loc[p]]
        box_plot_ls8 = axs[j, i].boxplot(data_to_plot, vert=1, widths=0.3, showfliers=False,
                      positions=positions,
                      labels=labels_ls8)
        for item in ['boxes', 'whiskers', 'caps']:
            plt.setp(box_plot_ls7[item], color='darkblue')
        axs[j, i].set_xticks(list(positions) + list(positions-0.4))
        axs[j, i].set_xticklabels(labels_ls8+labels_ls7,
                                  rotation=90, fontsize=8)
        axs[j, i].set_title(band)
        if i >= 2:
            j += 1
            i = 0
        else:
            i += 1
    plt.savefig('nbart_diff_plot/' + grid +'_nbar_alltime_diff.png', bbox_inches='tight')
    plt.close()

In [ ]:
sf_bands = ["blue", "green", "red", "nir", "swir1", "swir2"]
pcs = [np.around(a, 1) for a in np.arange(0.1, 1, 0.1)]
fig, axs = plt.subplots(2, 3,  sharey=True, sharex=True, figsize=(12, 8))
max_median_ls8 = {}
max_median_ls7 = {}
nbart_lin ={}
for band in sf_bands:
    max_median_ls8[band] = []
    max_median_ls7[band] = []
for grid in test_grids:
    grid = grid.lower()
    ls8_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls8_c3.csv')
    ls8_pc['time'] = ls8_pc['time'].astype(np.datetime64)
    ls8_pc = ls8_pc.set_index(['quantile', 'time'])
    ls7_pc = pd.read_csv('nbart_diff_data/'+grid+'_nbart_ls7_c3.csv')
    ls7_pc['time'] = ls7_pc['time'].astype(np.datetime64)
    ls7_pc = ls7_pc.set_index(['quantile', 'time'])
    for band in sf_bands:
        for p in pcs:
            max_median_ls8[band] += [ls8_pc.loc[p][band].median()]
            max_median_ls7[band] += [ls7_pc.loc[p][band].median()]

i = 0
j = 0
for band in sf_bands:
    nbart_lin[band] = linregress(max_median_ls8[band], max_median_ls7[band])
    axs[j, i].plot(max_median_ls8[band], max_median_ls7[band], 'o', color='SteelBlue',  mfc='none', markersize=5)
    axs[j, i].plot(max_median_ls8[band], nbart_lin[band].slope*np.array(max_median_ls8[band])+nbart_lin[band].intercept, color='darkorange',
                   label="regression")
    axs[j, i].axline([0, 0], [1, 1], color='darkgreen', ls='--', label="1:1")
    axs[j, i].set_title(band)
    axs[j, i].set_xlabel("LS8")
    axs[j, i].set_ylabel("LS7")
    if i >= 2:
        j += 1
        i = 0
    else:
        i += 1
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', ncol=1)
plt.tight_layout()
plt.savefig('nbart_diff_plot/' + 'nbart_regression.png', bbox_inches='tight')

In [ ]:
nbart_lin

In [86]:
chunks = {"y": -1, "x": -1}

for grid in test_grids:
    query_poly = poly_from_region_code(grid, "../../../au-grid.geojson")
    c3_query = {'geopolygon': query_poly}
    c3_query['time'] = ('2014-01-01', '2021-01-01')
    geobox = GeoBox.from_geopolygon(query_poly, (-30, 30), crs='epsg:3577')
    
    c3_ls8_datasets = dc.find_datasets(product='ga_ls8c_ard_3', **c3_query)
    c3_ls8 = load_with_native_transform(
        c3_ls8_datasets,
        bands=["blue", "green", "red", "nir", "swir1", "swir2", "fmask", "nbart_contiguity"],
        geobox=geobox,
        native_transform=_native_tr_nbart,
        fuser=_fuser_nbart,
        groupby="solar_day",
        resampling="nearest",
        chunks=chunks,
    )
    
    c3_land_raster = generate_seamask("../../../aus_map/cstauscd_r_3577.shp",
                                          c3_ls8.blue.shape[1:], (c3_ls8.x.data.min(), c3_ls8.y.data.max()), (30, -30))
        
    c3_ls8 = c3_ls8.where((c3_ls8 > -999) & c3_land_raster)
    for key, value in nbart_lin.items():
        c3_ls8[key] = c3_ls8[key] * value.slope + value.intercept
    c3_ls8 = c3_ls8.clip(min=0)
    ls8_pc_10 = c3_ls8.quantile(pcs, dim=['x', 'y'], skipna=True).compute().dropna('spec', how='all')
    ls8_pc_10 = ls8_pc_10.reset_index(['time', 'idx', 'uuid', 'grid'], drop=True).rename({'spec': 'time'}).to_dataframe()
    ls8_pc_10.to_csv('nbart_diff_data/' + grid.lower() + '_nbart_ls8_c3_lt.csv')

distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 16% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 15% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

In [ ]:
fig, axs = plt.subplots(figsize=(16, 9))

data_to_plot = []
for a, b in zip(ls7_box, ls8_box):
    data_to_plot += [a, b]
box_plot = plt.boxplot(data_to_plot, showfliers=False,
                  positions=[1, 1.6, 2.5, 3.1, 4, 4.6],
                  labels=['LS7-20','LS8-20','LS7-50','LS8-50','LS7-80','LS8-80'])

In [ ]:
# title too long for C2, drop spatial_ref: 3577
re_c2 = re_c2.drop_vars('spatial_ref')

In [ ]:
re_c2.PV_PC_10.loc[dict(time='2014-01-01')].where(c2_land_raster > 0).compute().plot(aspect=1.5, size=10)

In [ ]:
# plot the valid data for a band
(re_c2.PV_PC_10.loc[dict(time='2014-01-01')]-re_c2.PV_PC_10.loc[dict(time='2010-01-01')]).where(c2_land_raster > 0).compute().plot(aspect=1.5, size=10)
#plt.savefig('x45y17_2018_c2.png', bbox_inches='tight')

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** August 2021

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)

**Tags**: :index:`sandbox compatible`, :index:`landsat 8`, :index:`landsat 7`, :index: `landsat 5`, :index: `fc percentile`